In [1]:
from scipy import misc
import matplotlib.pyplot as plt

def split_into_fixed_kernels(imagefile, train_sample_size=32):
    img = misc.imread(imagefile)
    width = img.shape[0]
    height = img.shape[1]

    kernels = []
    for i in range(width // train_sample_size):
        for j in range(height // train_sample_size):
            x = i * train_sample_size
            y = j * train_sample_size
            kernels.append(img[x:x + train_sample_size, y:y + train_sample_size, 0:4])
    return kernels



In [2]:
before_kernels = split_into_fixed_kernels('imgs/christmas_1_b.png')
after_kernels = split_into_fixed_kernels('imgs/christmas_1_a.png')

/Users/phaser/Programe/anaconda/envs/HandsOnML/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  """


In [3]:
import numpy as np
import pandas as pd 

def dump_img_dataset(kernels, component, filename):
    to_be_written = []
    for i in range(len(kernels)):
        to_be_written.append(kernels[i][0:32, 0:32, component:component+1].flatten())
    
    df = pd.DataFrame(to_be_written)
    df.to_csv(filename + "_" + str(component) + ".csv", index=False)

dump_img_dataset(before_kernels, 0, 'before_pathtraced_image')
dump_img_dataset(before_kernels, 1, 'before_pathtraced_image')
dump_img_dataset(before_kernels, 2, 'before_pathtraced_image')

dump_img_dataset(after_kernels, 0, 'after_pathtraced_image')
dump_img_dataset(after_kernels, 1, 'after_pathtraced_image')
dump_img_dataset(after_kernels, 2, 'after_pathtraced_image')


In [4]:
import numpy as np
import pandas as pd

X_train = pd.read_csv('before_pathtraced_image_0.csv').as_matrix()
y_train = pd.read_csv('after_pathtraced_image_0.csv').as_matrix()

In [ ]:
import tensorflow as tf
import numpy as np

n_inputs = 32*32
n_hidden1 = 512
n_outputs = 32*32

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.float32, shape=(None, n_outputs), name="y")

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs + n_neurons)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z
        
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    logits = neuron_layer(hidden1, n_outputs, name="outputs")
    
with tf.name_scope("loss"):
    loss = tf.losses.absolute_difference(labels=y, predictions=logits)
    
learning_rate = 0.01
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.equal(logits, y)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

/Users/phaser/Programe/anaconda/envs/HandsOnML/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [ ]:
n_epochs = 40
batch_size = 50
print("test")
with tf.Session() as sess:
    init.run()
    print("Started...")
    for epoch in range(n_epochs):
        sess.run(training_op, feed_dict={X:X_train, y:y_train})
        acc_train = accuracy.eval(feed_dict={X:X_train, y:y_train})
        print(epoch, "Train accuracy:", acc_train)
    save_path = saver.save(sess, "./my_model_final.ckpt")

test
